In [ ]:
# Import required libraries
import pandas as pd
import re
import numpy as np

In [2]:
def estimate_fee_per_term(text):
    if pd.isna(text) or "ไม่พบข้อมูล" in text.lower():
        return np.nan
    if text.strip().startswith("http"):
        return np.nan

    numbers = [int(n.replace(",", "")) for n in re.findall(r'\d[\d,]*', text)]
    if not numbers:
        return np.nan

    text = text.lower()

    if re.fullmatch(r"\d[\d,]* *บาท?", text.strip()):
        fee = numbers[0]
        return round(fee / 8) if fee > 70000 else fee

    if re.fullmatch(r"\d[\d,]*", text.strip()):
        fee = numbers[0]
        return round(fee / 8) if fee > 70000 else fee

    if any(k in text for k in ["ต่อภาคการศึกษา", "ต่อภาคเรียน", "บาท/เทอม"]):
        if "ตลอดหลักสูตร" in text:
            bracket_match = re.search(r"\((.*?)\)", text)
            if bracket_match:
                inside = bracket_match.group(1).strip()
                pattern = r"(\d[\d,]*)\s*(บาท)?\s*ต่อ(ภาคเรียน|ภาคการศึกษา|เทอม)"
                inner_match = re.search(pattern, inside)
                if inner_match:
                    return int(inner_match.group(1).replace(",", ""))
        return numbers[0]

    if "ตลอดหลักสูตร" in text:
        return round(numbers[0] / 8)

    if "เทอมแรก" in text and "เทอมต่อไป" in text:
        if len(numbers) >= 2:
            return round((numbers[0] + numbers[1]*7) / 8)
        
    if len(numbers) == 2 and "แรกเข้า" in text:
        return round((numbers[0] * 8 + numbers[1]) / 8)

    if "ภาคการศึกษาละ 25,500" in text:
        return 25500

    if "ภาคการศึกษาแรก" in text and "ถัดไป" in text:
        if len(numbers) >= 2:
            return round((numbers[0] + numbers[1]*7) / 8)

    if "สายวิทยาศาสตร์" in text and "สายสังคม" in text:
        if len(numbers) >= 2:
            return round(sum(numbers[:2]) / 2)

    if len(numbers) == 1:
        return numbers[0]

    if len(numbers) == 2:
        return round(max(numbers) / 8)

    return np.nan

In [3]:
# Read the original data from Excel
df = pd.read_excel("tcas_data.xlsx")

# Filter rows for AI and Computer Engineering programs
df_ai = df[df["คำค้น"] == "วิศวกรรมปัญญาประดิษฐ์"]
df_ce = df[df["คำค้น"] == "วิศวกรรมคอมพิวเตอร์"]

# Find duplicate programs (same university and program name in both AI and CE)
duplicate_rows = pd.merge(
    df_ce,
    df_ai,
    on=["ชื่อมหาวิทยาลัย", "ชื่อหลักสูตร"],
    how="inner"
 )

# Get indices of duplicate rows in the Computer Engineering dataframe
drop_indices = df_ce[
    df_ce.set_index(["ชื่อมหาวิทยาลัย", "ชื่อหลักสูตร"]).index.isin(
        duplicate_rows.set_index(["ชื่อมหาวิทยาลัย", "ชื่อหลักสูตร"]).index
    )
].index

# Remove duplicate Computer Engineering rows from the original dataframe
df_cleaned = df.drop(index=drop_indices)

In [4]:
# Calculate per-term fee for each program using the custom function
df_cleaned["ค่าใช้จ่ายต่อภาคการศึกษา"] = df_cleaned["ค่าใช้จ่าย"].apply(estimate_fee_per_term)

# Separate programs with missing fee data
df_no_fee = df_cleaned[df_cleaned["ค่าใช้จ่ายต่อภาคการศึกษา"].isna()].copy()
df_no_fee = df_no_fee.drop(columns=["ค่าใช้จ่ายต่อภาคการศึกษา"])
df_no_fee["ค่าใช้จ่าย"] = df_no_fee["ค่าใช้จ่าย"].apply(
    lambda x: x if isinstance(x, str) and x.strip().startswith("http") else "ไม่พบข้อมูลค่าใช้จ่าย"
# If not a URL, mark as 'No fee data found' in Thai
)
df_no_fee.to_excel("tcas_no_fee.xlsx", index=False)
print("📁 Saved tcas_no_fee.xlsx for programs without fee data.")

# Keep only programs with valid per-term fee
df_final = df_cleaned.dropna(subset=["ค่าใช้จ่ายต่อภาคการศึกษา"]).copy()
df_final.to_excel("tcas_cleaned.xlsx", index=False)
print("✅ Removed duplicates and converted fees to per-term successfully!")

📁 Saved tcas_no_fee.xlsx for programs without fee data.
✅ Removed duplicates and converted fees to per-term successfully!
